In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
        process = psutil.Process(os.getpid())
        print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize(           process.memory_info().rss))
        print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=66ff6a2813ab9c19db456d5bb8d665936e660f56f1bc7bc56c8535c4923f4442
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.7 GB  | Proc size: 118.8 MB
GPU RAM Free: 15109MB | Used: 0MB | Util   0% | Total 15109MB


In [3]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
%%bash
cd /content
mkdir training_demo_50
cd /content/drive/MyDrive/ML-intern-project/tf2-logo-detection-pavan-IITB/workspace/training_demo_50/
cp -r annotations/ exported-models/ models/ pre-trained-models/ scripts *.py *.ipynb  /content/training_demo_50
cd /content

In [5]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2673, done.
remote: Counting objects: 100% (2673/2673), done.
remote: Compressing objects: 100% (2227/2227), done.
remote: Total 2673 (delta 675), reused 1334 (delta 414), pack-reused 0
Receiving objects: 100% (2673/2673), 32.65 MiB | 35.52 MiB/s, done.
Resolving deltas: 100% (675/675), done.


In [7]:
%%bash
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1650082 sha256=c72505c92a394657aecd5a6c550d60daacc64ba4287e62deb562037b060b9c71
  Stored in directory: /tmp/pip-ephem-wheel-cache-mc6xdzfs/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection
  Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


## Copying and cleaning data

In [ ]:
# from os import listdir
# import xml.etree.ElementTree as ET
# from PIL import Image

# output_dir = '/content/drive/MyDrive/RnD/TensorFlow/workspace/training_demo_50/images/'

# %cd /content/drive/MyDrive/RnD/images/train

# for f in listdir('/content/drive/MyDrive/RnD/images/train/'):
#     if f[-3:] == 'xml':
#         tree = ET.parse(f)
#         root = tree.getroot()
#         image_file_name = root.find('filename').text
#         image_name = image_file_name[:image_file_name.rfind(".")]
#         root.find('filename').text = image_name+'.jpg'
#         tree.write(output_dir + f)
#         im1 = Image.open(image_file_name).convert("RGB")
#         im1.save(output_dir + image_name + '.jpg', format='jpeg')


# %cd /content/drive/MyDrive/RnD/images/test

# for f in listdir('/content/drive/MyDrive/RnD/images/test/'):
#     if f[-3:] == 'xml':
#         tree = ET.parse(f)
#         root = tree.getroot()
#         image_file_name = root.find('filename').text
#         image_name = image_file_name[:image_file_name.rfind(".")]
#         root.find('filename').text = image_name+'.jpg'
#         tree.write(output_dir + f)
#         im1 = Image.open(image_file_name).convert("RGB")
#         im1.save(output_dir + image_name + '.jpg', format='jpeg')


- There were a lot of common images in train and test directories 
- Not all images have xmls 
- Some xmls were duplicated

In [ ]:
# total = []
# total1 = []
# for f in listdir(output_dir):
#     if f[-3:] == 'xml':
#         total.append(f[:f.rfind(".")])
#     else:
#         total1.append(f[:f.rfind(".")])

# l = list(set(total) - set(total1))

# # for i in l:
# #     os.remove(output_dir + i + '.xml')

In [ ]:
# print('Train Image:',1850, 'Train xml:',1833, 'Test Image:',400, 'Test Xml:',390, 'total Image:', 1850+400, 'Useful Image:', 2010)

In [ ]:
labels = '/content/drive/MyDrive/RnD/TensorFlow/workspace/training_demo_50/annotations/label_map.pbtxt'


## Partitioning the dataset 
using this code rather than partition_dataset file, so that each class is partitioned in balanced way

In [ ]:
# %cd /content/drive/MyDrive/RnD/TensorFlow/workspace/training_demo_50/images/

In [ ]:
# import os
# from sklearn.model_selection import train_test_split
# from shutil import copyfile

# output_dir = '/content/drive/MyDrive/RnD/TensorFlow/workspace/training_demo_50/images/'
# train_dir = output_dir + 'train/'
# test_dir = output_dir + 'test/'


# os.makedirs(train_dir)
# os.makedirs(test_dir)

# categories = []
# for f in listdir(output_dir):
#     if f[-3:] == 'xml':
#         tree = ET.parse(f)
#         root = tree.getroot()
#         categories.append(root.find('object').find('name').text)

# categories = list(set(categories))

# dict1 = {}
# for c in categories:
#     dict1[c] = []

# for f in listdir(output_dir):
#     if f[-3:] == 'xml':
#         tree = ET.parse(f)
#         root = tree.getroot()
#         image_name = root.find('filename').text
#         class_name = root.find('object').find('name').text
#         dict1[class_name].append([f,image_name])


# train_set =  []
# test_set = []

# for c in dict1:
#     train_set1, test_set1 = train_test_split(dict1[c], test_size=0.1)
#     train_set.extend(train_set1)
#     test_set.extend(test_set1)

# for f in train_set:
#     copyfile(f[0], train_dir+f[0])
#     copyfile(f[1], train_dir+f[1])

# for f in test_set:
#     copyfile(f[0], test_dir+f[0])
#     copyfile(f[1], test_dir+f[1])


In [ ]:
#%cd /content/drive/My Drive/RnD/TensorFlow/workspace/training_demo_total/scripts/preprocessing

In [ ]:
#!python partition_dataset.py -x -i  /content/drive/MyDrive/RnD/TensorFlow/workspace/training_demo_total/images -r 0.1

 Checking sbi images

In [ ]:
# output_dir = '/content/drive/MyDrive/RnD/TensorFlow/workspace/training_demo_50/images/'
# train_dir = output_dir + 'train/'

# from os import listdir
# import xml.etree.ElementTree as ET

# sbi_image_names = []
# sbi_xml_names = []

# for f in listdir(train_dir):
#     if f[-3:] == 'xml':
#         tree = ET.parse(train_dir+f)
#         root = tree.getroot()
#         image_name = root.find('filename').text
#         class_name = root.find('object').find('name').text
#         if class_name == 'State Bank of India':
#             sbi_image_names.append(image_name)
#             sbi_xml_names.append(f)
# print(len(sbi_image_names))

In [ ]:
# sbi_image_names = ['s (22).jpg', 's (24).jpg', 's (1).jpg', '20.jpg', 's (18).jpg', 's (12).jpg', '14.jpg', 's (14).jpg', 's (11).jpg', 
#                    's (25).jpg', 's (23).jpg', '30.jpg', 's (19).jpg', '26.jpg', 's (17).jpg', 's (9).jpg','s (10).jpg', 'n (11).jpg', 
#                    'n (12).jpg', 's (8).jpg', 's (28).jpg', '36.jpg', 'n (13).jpg', 's (15).jpg','s (4).jpg', 's (16).jpg', 'sbi_157.jpg', 
#                    'sbi_151.jpg', 'sbi_147.jpg', 'sbi_149.jpg', 'sbi_117.jpg', 'sbi_112.jpg', 'sbi_91.jpg', 'sbi_70.jpg', 'sbi_79.jpg']
# sbi_xml_names = ['s (22).xml', 's (24).xml', 's (1).xml', '20.xml', 's (18).xml', 's (12).xml', '14.xml', 's (14).xml', 's (11).xml', 
#                  's (25).xml', 's (23).xml', '30.xml', 's (19).xml', '26.xml', 's (17).xml', 's (9).xml', 's (10).xml', 'n (11).xml',
#                  'n (12).xml', 's (8).xml', 's (28).xml', '36.xml', 'n (13).xml', 's (15).xml', 's (4).xml', 's (16).xml', 'sbi_157.xml', 
#                  'sbi_151.xml', 'sbi_147.xml', 'sbi_149.xml', 'sbi_117.xml', 'sbi_112.xml', 'sbi_91.xml', 'sbi_70.xml', 'sbi_79.xml']
# print(len(sbi_xml_names))

In [ ]:
# print(len(sbi_image_names))

In [ ]:
# %matplotlib inline

In [ ]:
# import matplotlib.pyplot as plt
# from object_detection.utils import visualization_utils as viz_utils
# from PIL import Image
# import numpy as np
# from os import listdir
# import xml.etree.ElementTree as ET

# output_dir = '/content/drive/MyDrive/RnD/TensorFlow/workspace/training_demo_50/images/'
# train_dir = output_dir + 'train/'

# templ = [0,0,0]
# temp = 30
# image1 = Image.open(train_dir + sbi_image_names[temp])
# #print(sbi_xml_names[0])
# # (im_width, im_height) = image.size
# # image1 = np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
# tree = ET.parse(train_dir+sbi_xml_names[temp])
# root = tree.getroot()
# bndbox = root.find('object').find('bndbox')
# x1 = float(bndbox.find('xmin').text) + templ[0]
# y1 = float(bndbox.find('ymin').text) + templ[1]
# x2 = float(bndbox.find('xmax').text) + templ[2]
# y2 = float(bndbox.find('ymax').text)
# viz_utils.draw_bounding_box_on_image(image1, y1, x1, y2, x2, use_normalized_coordinates=False, thickness=2)
# plt.figure(figsize=(20, 20))
# plt.imshow(image1)

Editing the xmls to update the annotations. Major changes made are :-
1. Only considering the log and beside SBI or state bank of india 
2. That is ignoring below written text like online or any other text

In [ ]:
# change_xmax = [-30,-60,-79,-40,-108]
# change_xmin = [5,5,0,2,0]
# change_ymin = [0,0,0,0,7]

In [ ]:
# change_list = [12, 18, 22, 22, 10, 0, 8, 10, 15, 13, 15, 7, 20, 10, 11, 0, 0, 16, 13, 7, 0, 0, 0, 22, 0, 6, 0, 22, 0, 0, 6, 22]
# change_xmax = [-30,-60,-79,-40,-108]
# change_xmin = [5,5,0,2,0]
# change_ymin = [0,0,0,0,7]
# for i in range(len(sbi_xml_names)):
#     tree = ET.parse(train_dir+ sbi_xml_names[i])
#     root = tree.getroot()
#     bndbox = root.find('object').find('bndbox')
#     bndbox.find('ymin').text = str(int(bndbox.find('ymin').text) + change_ymin[i])
#     bndbox.find('xmin').text = str(int(bndbox.find('xmin').text) + change_xmin[i])
#     bndbox.find('xmax').text = str(int(bndbox.find('xmax').text) + change_xmax[i])
#     tree.write(train_dir+ sbi_xml_names[i])

## TfRecord generation

In [ ]:
# %cd /content/drive/My Drive/RnD/TensorFlow/workspace/training_demo_50/scripts/preprocessing

In [ ]:
# %%bash
# python generate_tfrecord.py \
# -x /content/drive/MyDrive/RnD/TensorFlow/workspace/training_demo_50/images/train \
# -l /content/drive/MyDrive/RnD/TensorFlow/workspace/training_demo_50/annotations/label_map.pbtxt \
# -o /content/drive/MyDrive/RnD/TensorFlow/workspace/training_demo_50/annotations/train.record \
# -i /content/drive/MyDrive/RnD/TensorFlow/workspace/training_demo_50/images/train

In [ ]:
# %%bash
# python generate_tfrecord.py \
# -x /content/drive/MyDrive/RnD/TensorFlow/workspace/training_demo_50/images/test \
# -l /content/drive/MyDrive/RnD/TensorFlow/workspace/training_demo_50/annotations/label_map.pbtxt \
# -o /content/drive/MyDrive/RnD/TensorFlow/workspace/training_demo_50/annotations/test.record

## Training with Resnet

In [ ]:
# %cd /content/drive/My Drive/RnD/TensorFlow/workspace/training_demo_50

In [ ]:
# !python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config

In [ ]:
#!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path models/my_ssd_resnet50_v1_fpn/pipeline.config --trained_checkpoint_dir models/my_ssd_resnet50_v1_fpn/ --output_directory exported-models/my_model_resnet


## Training with mobilenet

In [8]:
%cd /content/training_demo_50/

/content/training_demo_50


In [ ]:
!python model_main_tf2.py --model_dir=/content/drive/MyDrive/ML-intern-project/tf2-logo-detection-pavan-IITB/workspace/training_demo_50/models/my_ssd_mobilenet_v2_fpnlite --pipeline_config_path=models/my_ssd_mobilenet_v2_fpnlite/pipeline.config --alsologtostderr

2021-05-28 03:41:02.225112: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-05-28 03:41:04.509608: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-05-28 03:41:04.515832: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-28 03:41:04.516451: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-05-28 03:41:04.516490: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-05-28 03:41:04.529371: I tensorflow/stream_executor/platform/default

In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path models/my_ssd_mobilenet_v2_fpnlite/pipeline.config --trained_checkpoint_dir models/my_ssd_mobilenet_v2_fpnlite/ --output_directory exported-models/my_model_mobilenet_final

2021-05-24 03:51:39.744374: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Traceback (most recent call last):
  File "exporter_main_v2.py", line 98, in <module>
    import tensorflow.compat.v2 as tf
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/__init__.py", line 41, in <module>
    from tensorflow.python.tools import module_util as _module_util
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/__init__.py", line 46, in <module>
    from tensorflow.python import data
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/__init__.py", line 25, in <module>
    from tensorflow.python.data import experimental
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/experimental/__init__.py", line 99, in <module>
    from tensorflow.python.data.experimental import service
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/experimental/service/__in

In [ ]:
%%bash

cp -r /content/training_demo_50/exported-models/* /content/drive/MyDrive/ML-intern-project/tf2-logo-detection-pavan-IITB/workspace/training_demo_50/exported-models/
cp -r /content/training_demo_50/models/my_ssd_mobilenet_v2_fpnlite/* /content/drive/MyDrive/ML-intern-project/tf2-logo-detection-pavan-IITB/workspace/training_demo_50/models/my_ssd_mobilenet_v2_fpnlite/